In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
import yfinance as yf
from yahoofinancials import YahooFinancials
%matplotlib inline

In [ ]:
appl_df = yf.download('AAPL', 
                      start='2019-01-01', 
                      end='2020-12-31', 
                      progress=False)


Exception in thread Thread-9:
Traceback (most recent call last):
  File "/Users/kamranafzali/opt/anaconda3/envs/rain/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Users/kamranafzali/opt/anaconda3/envs/rain/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/kamranafzali/opt/anaconda3/envs/rain/lib/python3.8/site-packages/multitasking/__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "/Users/kamranafzali/opt/anaconda3/envs/rain/lib/python3.8/site-packages/yfinance/multi.py", line 166, in _download_one_threaded
    data = _download_one(ticker, start, end, auto_adjust, back_adjust,
  File "/Users/kamranafzali/opt/anaconda3/envs/rain/lib/python3.8/site-packages/yfinance/multi.py", line 178, in _download_one
    return Ticker(ticker).history(period=period, interval=interval,
  File "/Users/kamranafzali/opt/anaconda3/envs/rain/lib/python3.8/site-packages/yfinance/base.

In [ ]:
appl_df.head()

In [ ]:
appl_df['Open'].plot(title="Apple's stock price")

In [ ]:
appl_df['Open']=appl_df['Open'].pct_change()
appl_df['Open'].plot(title="Apple's stock return")

In [ ]:
sc = StandardScaler()

In [ ]:
def preproc( data, lag, ratio):
    data=data.dropna().iloc[:, 0:1]
    Dates=data.index.unique()
    data.iloc[:, 0] = sc.fit_transform(data.iloc[:, 0].values.reshape(-1, 1))
    for s in range(1, lag):
        data['shift_{}'.format(s)] = data.iloc[:, 0].shift(s)
    X_data = data.dropna().drop(['Open'], axis=1)
    y_data = data.dropna()[['Open']]
    index=int(round(len(X_data)*ratio))
    X_data_train=X_data.iloc[:index,:]
    X_data_test =X_data.iloc[index+1:,:]
    y_data_train=y_data.iloc[:index,:]
    y_data_test =y_data.iloc[index+1:,:]
    return X_data_train,X_data_test,y_data_train,y_data_test,Dates;

In [ ]:
a,b,c,d,e=preproc(appl_df, 25, 0.90)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import keras.backend as K
from keras.callbacks import EarlyStopping

In [ ]:
import keras_tuner as kt
from tensorflow import keras

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(hp.Choice('units', [8, 16, 32]), activation='relu'))
    model.add(keras.layers.Dense(1, activation='relu'))
    model.compile(loss='mse')
    return model

In [ ]:
tuner = kt.RandomSearch( build_model, objective='val_loss', max_trials=5)

In [ ]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

In [ ]:
best_model = tuner.get_best_models()[0]